In [1]:
import sys
import os
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import oandapyV20.endpoints.instruments as instruments
import plotly.graph_objects as go
from src.fetch_historical_data import FetchHistoricalData
from src.q_learning import QLearningTrader
from src.utils import calculate_indicators
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
token = os.getenv("OANDA_ACCESS_TOKEN")

In [3]:
data = FetchHistoricalData('USD_JPY', 'M1', token, 5000).fetch_and_process_data()
data = calculate_indicators(data).dropna(inplace=False)

In [4]:
# train test split by index
train = data.iloc[:int(0.8*len(data))]
test = data.iloc[int(0.8*len(data)):]

In [5]:
qtrader = QLearningTrader(
            num_actions=3,
            num_features=11,
            learning_rate=0.01,
            discount_factor=0.9,
            exploration_prob=0.1,
        )

In [6]:
train

,High,Close,Low,Open,SMA,RSI,MACD,%K,%D,resistance,support
Time,,,,,,,,,,,
2024-04-09 17:22:00+08:00,151.868,151.856,151.856,151.866,151.8630,11.656240,-0.001530,8.333333,31.944444,151.865000,151.861000
2024-04-09 17:23:00+08:00,151.858,151.854,151.853,151.856,151.8606,9.399894,-0.002345,5.882353,20.016340,151.863188,151.858012
2024-04-09 17:24:00+08:00,151.866,151.856,151.855,151.855,151.8590,29.787015,-0.002361,20.000000,11.405229,151.861550,151.856450
2024-04-09 17:25:00+08:00,151.861,151.858,151.854,151.858,151.8578,47.505673,-0.001933,33.333333,19.738562,151.859933,151.855667
2024-04-09 17:26:00+08:00,151.868,151.864,151.858,151.860,151.8576,75.419417,-0.000621,73.333333,42.222222,151.859524,151.855676
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-12 12:16:00+08:00,153.229,153.226,153.225,153.225,153.2248,85.974243,0.002972,57.142857,73.214286,153.226142,153.223458
2024-04-12 12:17:00+08:00,153.226,153.224,153.222,153.226,153.2256,45.736219,0.002478,33.333333,50.992063,153.226047,153.225153
2024-04-12 12:18:00+08:00,153.228,153.226,153.224,153.224,153.2256,68.118317,0.002458,40.000000,43.492063,153.226047,153.225153


In [7]:
actions, cumulative_rewards = qtrader.train(train)

2024-04-14 19:39:39.281 | INFO     | src.q_learning:train:106 - Training the Q-learning model...
2024-04-14 19:39:39.777 | INFO     | src.q_learning:train:145 - Training complete.


State: 
151.856, Action: 0, Reward: -1.3170371931179065e-05, Updated Q-value: None, Cumulative reward: -1.3170371931179065e-05
State: 
151.854, Action: 0, Reward: 1.3170545392160415e-05, Updated Q-value: 1.3170545392160414e-07, Cumulative reward: 1.7346098135065293e-10
State: 
151.856, Action: 0, Reward: 1.3170371931366227e-05, Updated Q-value: 1.3170371931366227e-07, Cumulative reward: 1.3170545392347577e-05
State: 
151.858, Action: 0, Reward: 3.9510595424674544e-05, Updated Q-value: 5.266779698410941e-07, Cumulative reward: 5.2681140817022125e-05
State: 
151.864, Action: 0, Reward: -0.0006453142285203418, Updated Q-value: -5.926990993332164e-06, Cumulative reward: -0.0005926330877033196
State: 
151.766, Action: 1, Reward: -0.00013178182201553862, Updated Q-value: -7.185539303554229e-06, Cumulative reward: -0.0007244149097188582
State: 
151.786, Action: 0, Reward: 7.247045182029753e-05, Updated Q-value: 7.258898672882698e-07, Cumulative reward: -0.0006519444578985607
State: 
151.797, 

In [10]:
train_plot = train.copy()
train_plot['Action'] = actions + [np.nan]
train_plot['Cumulative Rewards'] = cumulative_rewards
train_plot['Individual Rewards'] = train_plot['Cumulative Rewards'].diff()
train_plot['Individual Rewards'].iloc[1] = train_plot['Cumulative Rewards'].iloc[1]

In [12]:
train_plot.head(10)

,High,Close,Low,Open,SMA,RSI,MACD,%K,%D,resistance,support,Action,Cumulative Rewards,Individual Rewards
Time,,,,,,,,,,,,,,
2024-04-09 17:22:00+08:00,151.868,151.856,151.856,151.866,151.8630,11.656240,-0.001530,8.333333,31.944444,151.865000,151.861000,0.0,NaN,NaN
2024-04-09 17:23:00+08:00,151.858,151.854,151.853,151.856,151.8606,9.399894,-0.002345,5.882353,20.016340,151.863188,151.858012,0.0,-1.317037e-05,-0.000013
2024-04-09 17:24:00+08:00,151.866,151.856,151.855,151.855,151.8590,29.787015,-0.002361,20.000000,11.405229,151.861550,151.856450,0.0,1.734610e-10,0.000013
2024-04-09 17:25:00+08:00,151.861,151.858,151.854,151.858,151.8578,47.505673,-0.001933,33.333333,19.738562,151.859933,151.855667,0.0,1.317055e-05,0.000013
2024-04-09 17:26:00+08:00,151.868,151.864,151.858,151.860,151.8576,75.419417,-0.000621,73.333333,42.222222,151.859524,151.855676,0.0,5.268114e-05,0.000040
2024-04-09 17:27:00+08:00,151.866,151.766,151.728,151.864,151.8396,5.376417,-0.016163,27.142857,44.603175,151.860257,151.818943,1.0,-5.926331e-04,-0.000645
2024-04-09 17:28:00+08:00,151.800,151.786,151.758,151.760,151.8260,26.322831,-0.020706,41.428571,47.301587,151.849141,151.802859,0.0,-7.244149e-04,-0.000132
2024-04-09 17:29:00+08:00,151.800,151.797,151.784,151.787,151.8142,37.700400,-0.020340,49.285714,39.285714,151.836298,151.792102,1.0,-6.519445e-04,0.000072
2024-04-09 17:30:00+08:00,151.798,151.798,151.774,151.796,151.8022,38.985243,-0.018951,50.000000,46.904762,151.820634,151.783766,1.0,-6.585322e-04,-0.000007


In [13]:
train_plot["Drawdown"] = train_plot["Cumulative Rewards"] - train_plot["Cumulative Rewards"].cummax()

In [14]:
fig = go.Figure(data=[go.Candlestick(x=train.index,
                open=train_plot["Open"],
                high=train_plot["High"],
                low=train_plot["Low"],
                close=train_plot["Close"])])

fig.update_layout(
    title="Training data",
    xaxis=dict(
        rangeslider=dict(
            visible=False
        )
    )
)

fig.show()

In [15]:
action_dict = {0: 'Buy', 1: 'Sell', 2: 'Hold'}
train_plot['Action'] = train_plot['Action'].map(action_dict)

fig = go.Figure()

# Add cumulative reward trace
fig.add_trace(go.Scatter(x=train_plot.index, y=train_plot["Cumulative Rewards"], mode='lines', name='Cumulative Rewards', line=dict(width=4)))

# Add action traces
for action in train_plot["Action"].unique():
    mask = train_plot["Action"] == action
    fig.add_trace(go.Scatter(x=train_plot[mask].index, y=train_plot[mask]["Cumulative Rewards"], mode='markers', name=f'Action: {action}', marker=dict(size=6)))

fig.update_layout(title="Cumulative rewards for training set")
fig.show()

In [16]:
# plot the drawdown 
fig = go.Figure()

fig.add_trace(go.Scatter(x=train_plot.index, y=train_plot["Drawdown"], mode='lines', name='Drawdown', line=dict(width=4)))

fig.update_layout(title="Drawdown for training set")
fig.show()

In [20]:
num_periods_per_year = 24 * 60 * 5 * 52  # 24 hours, 60 minutes, 5 days a week, 52 weeks a year

annualized_mean = train_plot["Individual Rewards"].mean() * num_periods_per_year
annualized_std = train_plot["Individual Rewards"].std() * np.sqrt(num_periods_per_year)

annualized_mean / annualized_std

6.271510267415746

In [22]:
annualized_std

0.05964633041273261

In [21]:
annualized_mean

0.3740725735971246

In [23]:
for idx, row in test.iterrows():
    new_data_df = pd.DataFrame(row).T
    qtrader.update(train, new_data_df)
    train =  pd.concat([train, new_data_df], ignore_index=True)

Hold signal detected.
State: 
High          153.227000
Close         153.224000
Low           153.218000
Open          153.223000
SMA           153.224400
RSI            50.837398
MACD            0.001366
%K             50.000000
%D             30.000000
resistance    153.225237
support       153.223563
Name: 2024-04-12 12:20:00+08:00, dtype: float64, Action: 2, Reward: 6.526392732240216e-06, Updated Q-value: 1.0884807380017473e-05, Cumulative reward: 6.526392732240216e-06
Hold signal detected.
State: 
High          153.225000
Close         153.225000
Low           153.222000
Open          153.224000
SMA           153.224200
RSI            59.699023
MACD            0.001376
%K             58.333333
%D             36.111111
resistance    153.224942
support       153.223458
Name: 3995, dtype: float64, Action: 2, Reward: -1.3052700277246716e-05, Updated Q-value: 1.0743395569864989e-05, Cumulative reward: -1.3052700277246716e-05
Hold signal detected.
State: 
High          153.225000
Close 